In [68]:
# téléchargement

from pymatgen.ext.matproj import MPRester
import pymatgen.electronic_structure.core as elstruct
from matplotlib import pyplot as plt

cle_materiau = "mp-20072"
cle_utilisateur = "om6hvHlirPcyuxXF"

with MPRester(cle_utilisateur) as m:
    band = m.get_bandstructure_by_material_id(cle_materiau)

In [81]:
# petite fonction utilitaire avec exemple

def get_indexes(a, number=1,maxi=True):
    """
    donne dans un tableau les indices des 'numbers' premiers maximums du tableau 'a'
    par défaut, la fonction renvoie le maximum du tableau (numbers = 1)
    
    Si max est mis à 'False', la fonction ne cherche plus les indices des maximus
    mais ceux des minimums
    """
    index = [-1]*number
    if maxi:
        init_value = min(a)
        condition = lambda a,b : a > b
    else:
        init_value = max(a)
        condition = lambda a,b : a < b
    
    for i in range(number):
        max_min = init_value
        index_i = -1
        for j in range(len(a)):
            if condition(a[j],max_min)  and j not in index:
                max_min = a[j]
                index_i = j
        index[i] = index_i
    
    return index
    

# exemple 

a = [0,500,8,-5,12]
print('si a = '+str(a)+' et numbers = 2,')
print('le résultat est : '+str(get_indexes(a,2)))
print('ce qui donne bien les indices des 2 plus grandes valeurs du tableau')
print('\nsi on fait l\'opération avec max à False, on a : '+str(get_indexes(a,2,False)))
print('ce qui donne bien les indices des 2 plus petites valeurs du tableau')

si a = [0, 500, 8, -5, 12] et numbers = 2,
le résultat est : [1, 4]
ce qui donne bien les indices des 2 plus grandes valeurs du tableau

si on fait l'opération avec max à False, on a : [3, 0]
ce qui donne bien les indices des 2 plus petites valeurs du tableau


In [83]:
# on récupère la bande interdite et on imprime ses attributs
band_gap = band.get_band_gap()
print('La bande interdite est directe ?     : '+str(band_gap['direct']))
print('L\'énergie de la bande interdite est  : '+str(round(band_gap['energy'],4))+' [V]')
print('transition de la bande interdite     : '+band_gap['transition'])


# on recupere les donnees pour les premieres et dernieres bandes

# The energy eigenvalues as a {spin: ndarray}. Note that the use of an ndarray is necessary
# for computational as well as memory efficiency due to the large amount of numerical data. 
# The indices of the ndarray are [band_index, kpoint_index]

cle = elstruct.Spin(1)
premiere_bande = band.bands[cle][0]
derniere_bande = band.bands[cle][-1]

for bande in [premiere_bande, derniere_bande]:
    index_max = get_indexes(bande)
    index_min = get_indexes(bande,maxi=False)
    print(bande[index_min])
    print(bande[index_max])

La bande interdite est directe ?     : False
L'énergie de la bande interdite est  : 2.5162 [V]
transition de la bande interdite     : Z-L
[-64.4796]
[-64.4791]
[17.7439]
[20.8395]
